In [1]:
#Importing file
import tarfile
import os

#Shuffling
import random

#Data cleaning
import re
import string


import numpy as np
import pandas as pd

#Tokenize and model
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer


In [2]:
nltk.download('stopwords')
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\knmeh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\knmeh\AppData\Roaming\nltk_data...


True

In [3]:
def clean_text(text):
#will replace the html characters with " "
    text=re.sub('<.*?>', ' ', text)  
    #To remove the punctuations
    text = text.translate(str.maketrans(' ',' ',string.punctuation))
    #will consider only alphabets and numerics
    text = re.sub('[^a-zA-Z]',' ',text)  
    #will replace newline with space
    text = re.sub("\n"," ",text)
    #will convert to lower case
    text = text.lower()
    # will split and join the words
    text=' '.join(text.split())
    return text

In [4]:
stop_words = set(stopwords.words('english'))
remove_stop_words =  lambda text: ' '.join([word for word in text.split() if word not in (stop_words)])



In [ ]:
file = tarfile.open('../input/imdbdataset/imdb.tgz')
  
file.extractall('./imdb')
  
file.close()

In [9]:
pathneg='C:/XXX/imdb/train/neg'
pathpos='C:/XXXX/imdb/train/pos'

In [12]:
neg=[]
pos=[]
for x in os.listdir(pathneg):
    f= open(pathneg + '/' + x, encoding="utf8")
    neg.append(f.read())

for x in os.listdir(pathpos):
    f= open(pathpos + '/' + x, encoding="utf8")
    pos.append(f.read())

In [ ]:
print(neg[0],pos[0])

In [13]:
neglabels = [0 for x in neg]
poslabels = [1 for x in pos]

In [14]:
total_sentences = neg + pos
total_labels = neglabels + poslabels

In [15]:
for i in range(len(total_sentences)):
    total_sentences[i] = total_sentences[i] + str(total_labels[i])

In [16]:
random.shuffle(total_sentences)
total_labels = [int(total_sentences[i][-1]) for i in range(len(total_sentences))]

In [17]:
# Removing label
total_sentences= [i.rstrip(i[-1]) for i in total_sentences]
total_sentences[0]

'This is a pretty bad movie. The plot is sentimental mush. I suppose the production values are OK, decent photography, unobtrusive direction and all that. Mark Hamill was terrible. I\'ve never cared much for him, and this movie validates that perception. It\'s no wonder that he never really had any sort of career aside from his "Star Wars" films. I\'ll just say "Buh-bye, Mark" as he sinks into well-deserved obscurity. On the other hand, a very young Annie Potts utterly stole the show. She showed charm, beauty, and acting chops all in one performance. I remember seeing her in "Texasville" recently and taking note of her beauty. It\'s interesting seeing her from ten years earlier. Anyway, unless you are interested in Miss Potts, run away from this film screaming for the hills.'

In [18]:
total_sentences = [clean_text(i) for i in total_sentences]
total_sentences[0]

'this is a pretty bad movie the plot is sentimental mush i suppose the production values are ok decent photography unobtrusive direction and all that mark hamill was terrible ive never cared much for him and this movie validates that perception its no wonder that he never really had any sort of career aside from his star wars films ill just say buhbye mark as he sinks into welldeserved obscurity on the other hand a very young annie potts utterly stole the show she showed charm beauty and acting chops all in one performance i remember seeing her in texasville recently and taking note of her beauty its interesting seeing her from ten years earlier anyway unless you are interested in miss potts run away from this film screaming for the hills'

In [19]:
total_sentences = [ remove_stop_words(i) for i in total_sentences]
total_sentences[0]

'pretty bad movie plot sentimental mush suppose production values ok decent photography unobtrusive direction mark hamill terrible ive never cared much movie validates perception wonder never really sort career aside star wars films ill say buhbye mark sinks welldeserved obscurity hand young annie potts utterly stole show showed charm beauty acting chops one performance remember seeing texasville recently taking note beauty interesting seeing ten years earlier anyway unless interested miss potts run away film screaming hills'

In [20]:
words = []
for i in total_sentences:
    words.extend(i.split(" "))

In [ ]:
bigrams = (pd.Series(nltk.ngrams(words, 2)).value_counts())
trigrams = (pd.Series(nltk.ngrams(words, 3)).value_counts())

In [ ]:
bigrams[0:15]

Most of these are superlatives and that should make determining sentiment easier

In [ ]:
trigrams[0:15]

In [21]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
sia.polarity_scores("""i have no idea what these people were thinking when they made this film no plot very limited
 action and what is with the rd person commentary throughout the film instead of running around the planet to shoot on all 
 of these locations they should have spent some money on script writing and actors what acting there was was lousy this was minutes
  of my life i will never be able to get back i should bill the director for the cost of renting this film to the director and 
  the writers of this filmplease quit now this film should have a tag on the front of it saying beware of boredom the only
 good thing i can say about this film is the computer generation its ok as generation is this movie should never have a sequelever""")

{'neg': 0.103, 'neu': 0.862, 'pos': 0.035, 'compound': -0.8268}

Not very Reliable since this is clearly very negative

In [22]:
X_train = total_sentences
y_train= total_labels
vec = CountVectorizer(stop_words='english')
X_train = vec.fit_transform(X_train).toarray()
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [23]:
model.predict(vec.transform(["""
This is an old-fashioned movie that adheres to admittedly familiar principles of storytelling and emotional stakes, 
but by way of such a winning cast, evocative atmosphere and genuine tone that its impossible not to love. 
For audiences weary of superheroic bombast and worn out from puzzling through art house arcana, 
             “CODA” is here to save the day. It’s sweet, funny, meaningful and accessible in precisely the right measure.
"""]))

array([1])

In [ ]:
X_train.shape

In [29]:
negtest=[]
postest=[]
negpath= 'C:/XXXX/imdb/test/neg/'
for x in os.listdir(negpath):
    f= open(negpath+ x,encoding = "utf-8")
    negtest.append(f.read())
pospath='C:/XXXX/imdb/test/pos/'
for x in os.listdir(pospath):
    f= open(pospath + x,encoding = "utf-8")
    postest.append(f.read())

In [30]:
negtestlabels = [0 for x in negtest]
postestlabels = [1 for x in postest]

In [31]:
test_sentences = negtest + postest
test_labels = negtestlabels + postestlabels

In [32]:
test_sentences = [clean_text(i) for i in test_sentences]
test_sentences = [ remove_stop_words(i) for i in test_sentences]

In [34]:
X_test = test_sentences[12300:13500]
y_test= test_labels[12300:13500]
X_test =  vec.transform(X_test).toarray()
preds=model.predict(X_test)

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, preds)

0.81

In [ ]:
We only used 12300:13500 because test dataset has 25000 samples and 